<a href="https://colab.research.google.com/github/sneharreddy25/project_25-/blob/main/Data_science_assesment_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 11.6 MB/s eta 0:00:00


In [ ]:
%%writefile rag.py
def get_rag_answer(query: str) -> str:
    """
    Placeholder function for RAG answer generation.
    Replace this with your actual RAG implementation.
    """
    return f"This is a placeholder answer for your query: '{query}'. Please implement your RAG logic in rag.py."

Writing rag.py


In [ ]:
pip install nest_asyncio

In [ ]:
import logging
import nest_asyncio

BOT_TOKEN = "YOUR_TELEGRAM_BOT_TOKEN"

logging.basicConfig(level=logging.INFO)

# Apply the nest_asyncio patch at the very beginning of your application
nest_asyncio.apply()
logging.info("nest_asyncio patch applied.")

In [ ]:
from telegram import Update
from telegram.ext import ContextTypes

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Welcome! 🤖\n\n"
        "Use:\n"
        "/ask <question> - Ask from knowledge base\n"
        "/help - Show help"
    )

In [ ]:
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "/ask <query> - Ask a question\n"
        "/help - Show usage instructions"
    )



In [ ]:
async def ask(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = " ".join(context.args)
    if not query:
        await update.message.reply_text("Please provide a query.")
        return

    answer = get_rag_answer(query)
    await update.message.reply_text(answer)



In [ ]:
from telegram.ext import ApplicationBuilder, CommandHandler

def main():
    # The previous workaround for event loop management is no longer needed
    # with nest_asyncio applied globally.

    app = ApplicationBuilder().token(BOT_TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(CommandHandler("ask", ask))

    app.run_polling()

In [ ]:
if __name__ == "__main__":
     main()

ERROR:telegram.ext:Network Retry Loop (Bootstrap Initialize Application): Invalid token. Aborting retry loop.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telegram/_bot.py", line 854, in initialize
    await self.get_me()
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_extbot.py", line 1999, in get_me
    return await super().get_me(
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/_bot.py", line 985, in get_me
    result = await self._post(
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/_bot.py", line 703, in _post
    return await self._do_post(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_extbot.py", line 372, in _do_post
    return await super()._do_post(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/_bot.py", line 732, in _do_post
    result = await reques

RuntimeError: Cannot close a running event loop

In [ ]:
from sentence_transformers import SentenceTransformer
import sqlite3
import numpy as np
import openai

openai.api_key = "YOUR_OPENAI_KEY"

EMBED_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
DB_PATH = "embeddings.db"

def embed_text(text):
    return EMBED_MODEL.encode(text)

def get_top_chunks(query, k=3):
    query_vec = embed_text(query)
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.execute("SELECT content, embedding FROM docs")
    rows = cur.fetchall()

    scored = []
    for content, emb in rows:
        emb = np.frombuffer(emb, dtype=np.float32)
        score = np.dot(query_vec, emb)
        scored.append((score, content))

    conn.close()
    scored.sort(reverse=True)
    return [c for _, c in scored[:k]]

def get_rag_answer(query):
    context_chunks = get_top_chunks(query)
    context = "\n".join(context_chunks)

    prompt = f"""
    Answer the question based on the context below:

    Context:
    {context}

    Question:
    {query}
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import sqlite3
import os
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
conn = sqlite3.connect("embeddings.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS docs (
    id INTEGER PRIMARY KEY,
    content TEXT,
    embedding BLOB
)
""")

DOCS_DIR = "docs"

for file in os.listdir(DOCS_DIR):
    with open(os.path.join(DOCS_DIR, file), "r") as f:
        text = f.read()
        emb = model.encode(text).astype(np.float32).tobytes()
        cur.execute("INSERT INTO docs (content, embedding) VALUES (?,?)", (text, emb))

conn.commit()
conn.close()
print("✅ Documents ingested successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'docs'

In [ ]:
import os

DOCS_DIR = "docs"
if not os.path.exists(DOCS_DIR):
    os.makedirs(DOCS_DIR)
    print(f"Created directory: {DOCS_DIR}")
else:
    print(f"Directory already exists: {DOCS_DIR}")

print("Now, you can add your text documents inside the 'docs' folder. After adding them, re-run the document ingestion cell (ekgAgHyjb5Ma).")

Created directory: docs
Now, you can add your text documents inside the 'docs' folder. After adding them, re-run the document ingestion cell (ekgAgHyjb5Ma).


In [ ]:
import sqlite3
import os
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
conn = sqlite3.connect("embeddings.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS docs (
    id INTEGER PRIMARY KEY,
    content TEXT,
    embedding BLOB
)
""")

DOCS_DIR = "docs"

for file in os.listdir(DOCS_DIR):
    with open(os.path.join(DOCS_DIR, file), "r") as f:
        text = f.read()
        emb = model.encode(text).astype(np.float32).tobytes()
        cur.execute("INSERT INTO docs (content, embedding) VALUES (?,?)", (text, emb))

conn.commit()
conn.close()
print("✅ Documents ingested successfully")



✅ Documents ingested successfully


#  Mini-RAG Telegram Bot

A lightweight GenAI Telegram bot that answers user questions using a
Retrieval-Augmented Generation (RAG) pipeline.

---

##  Features
- Telegram bot interface
- Semantic search over local documents
- Retrieval-Augmented Generation
- Fast and lightweight
- Local embedding storage with SQLite

---

## 🏃 How to Run Locally

### 1️⃣ Install Dependencies
```bash
pip install python-telegram-bot sentence-transformers openai numpy


Add Documents

Place 3–5 .txt or .md files inside the docs/ directory.

3️⃣ Ingest Documents
python ingest.py

4️⃣ Run the Bot
python app.py

🧠 Models & APIs Used

Embedding Model: all-MiniLM-L6-v2 (SentenceTransformers)

Vector Store: SQLite

LLM: OpenAI GPT-3.5 Turbo

Bot Framework: python-telegram-bot
